In [16]:
import pandas as pd 
from typing import List, Dict, Any
import numpy as np

In [2]:
exp1 = "/root/tommaso/Variety/LLM-Inversion/results/ALL_complete_meaningful_2.csv"
exp2 = "/root/tommaso/Variety/LLM-Inversion/results/ALL_complete_meaningful.csv"
exp3 = "/root/tommaso/Variety/LLM-Inversion/results/Baseline_20_meaningful.csv"
exp4 = "/root/tommaso/Variety/LLM-Inversion/results/Baseline_20_random.csv"
exp5 = "/root/tommaso/Variety/LLM-Inversion/results/SGD_20_random.csv"
exp6 = "/root/tommaso/Variety/LLM-Inversion/results/SGD_complete_meaningful.csv"

In [3]:
df1 = pd.read_csv(exp1)
df2 = pd.read_csv(exp2)
df3 = pd.read_csv(exp3)
df4 = pd.read_csv(exp4)
df5 = pd.read_csv(exp5)
df6 = pd.read_csv(exp6)

In [20]:
# concat df1 and df2 only 
df12 = pd.concat([df1, df2], ignore_index=True)
# remove element with optimizer column == "LBFGS"
df12 = df12[df12["optimizer"] != "LBFGS"]

In [4]:
df1.head()

,dataset,index,layer,learning_rate,optimizer,token_length,match,inversion_time,timesteps,times
0,token_length_filtered_dataset.csv,0,4,0.01,Adam,20,True,20.923944,88_134_77_68_113_119_115_83_69_58_107_132_270_...,0.91_1.31_0.75_0.67_1.10_1.16_1.13_0.81_0.68_0...
1,token_length_filtered_dataset.csv,1,4,0.01,Adam,20,True,23.161463,71_43_89_215_187_420_104_28_32_106_140_76_116_...,0.66_0.42_0.87_2.08_1.78_4.03_0.96_0.26_0.29_0...
2,token_length_filtered_dataset.csv,2,4,0.01,Adam,20,True,18.770709,71_40_130_126_116_105_122_69_66_152_31_118_113...,0.62_0.37_1.19_1.16_1.07_0.97_1.12_0.63_0.61_1...
3,token_length_filtered_dataset.csv,3,4,0.01,Adam,20,True,18.173630,30_319_199_19_39_127_62_99_33_40_54_101_108_14...,0.26_2.92_1.83_0.17_0.36_1.17_0.57_0.91_0.30_0...
4,token_length_filtered_dataset.csv,4,4,0.01,Adam,20,True,21.562065,30_38_177_67_35_109_88_171_124_122_158_141_130...,0.27_0.35_1.65_0.62_0.33_1.02_0.82_1.60_1.16_1...


In [24]:
def compute_time_steps(df, last_timestep: List[int] = [10], type: str = "timesteps"):
    assert type in ["timesteps", "times"], "give a valid type"
    column = df[type]
    scores = column.apply(lambda x: list(float(i) for i in x.split("_")))
    # initialize an empty dictionary to store the results
    results: Dict[str, Any] = {}
    # for loop in the time steps and compute the mean and std score up to that point, save it in a dictionary with keys mean_{type}_n std...
    for i in last_timestep:
        mean_score = scores.apply(lambda x: np.mean(x[:i+1])).mean()
        std_score = scores.apply(lambda x: np.std(x[:i+1])).mean()
        # mean_score = scores.apply(lambda x: sum(x[:i + 1]) / (i + 1)).mean()
        # std_score = scores.apply(lambda x: (sum((xi - mean_score) ** 2 for xi in x[:i + 1]) / (i + 1)) ** 0.5).mean()
        results[f"mean_{type}_{i}"] = mean_score
        results[f"std_{type}_{i}"] = std_score
    return results

In [25]:
# print columns with uniques values 
def print_unique_columns(df):
    for column in df.columns:
        unique_values = df[column].unique()
        print(f"Column: {column}, Unique values: {unique_values}")


# Plotting Experiment 2 

In [26]:
print_unique_columns(df12)

Column: dataset, Unique values: ['token_length_filtered_dataset.csv']
Column: index, Unique values: [0 1 2 3 4 5 6 7 8 9]
Column: layer, Unique values: [4]
Column: learning_rate, Unique values: [0.01 1.  ]
Column: optimizer, Unique values: ['Adam' 'AdamW' 'RMSprop']
Column: token_length, Unique values: [20]
Column: match, Unique values: [ True]
Column: inversion_time, Unique values: [ 20.92394352  23.16146302  18.77070904  18.17362976  21.56206465
  18.08984542  19.00598383 133.53544545  19.87070227  15.57244444
  20.49819398  23.43648076  19.46755981  18.70466161  22.06325674
  18.39428163  18.75789785 125.87159395  19.64991283  15.32918692
   9.786731    12.05488276   8.40075135   9.27157784  11.47732067
   7.05476165   8.26860595 125.61394978   9.10261989  80.78740621
   2.63861823   2.52050972   2.07424045   2.8558774    2.91652989
   2.22158504   2.51750231 213.06782103   3.25891304 117.56600642
   2.66802812   2.52497172   2.04329276   2.87625432   3.31027627
   2.23083639   2.51

In [29]:
layer = 4
learning_rate = 0.01
optimizer_list = ['Adam', 'AdamW', 'RMSprop']
type = "timesteps"
results_4_001 = {}

for optimizer in optimizer_list:
    df_filtered = df12[(df12["layer"] == layer) & (df12["learning_rate"] == learning_rate) & (df12["optimizer"] == optimizer)]
    result = compute_time_steps(df_filtered, type=type, last_timestep=list(range(20)))
    print(f"Results for layer {layer}, learning rate {learning_rate}, optimizer {optimizer}:")
    print(result)


Results for layer 4, learning rate 0.01, optimizer Adam:
{'mean_timesteps_0': np.float64(50.3), 'std_timesteps_0': np.float64(0.0), 'mean_timesteps_1': np.float64(86.5), 'std_timesteps_1': np.float64(42.1), 'mean_timesteps_2': np.float64(98.03333333333333), 'std_timesteps_2': np.float64(47.56022793113632), 'mean_timesteps_3': np.float64(96.375), 'std_timesteps_3': np.float64(51.310030647698355), 'mean_timesteps_4': np.float64(93.20000000000002), 'std_timesteps_4': np.float64(50.242646780429155), 'mean_timesteps_5': np.float64(98.11666666666666), 'std_timesteps_5': np.float64(55.95389228848838), 'mean_timesteps_6': np.float64(99.38571428571429), 'std_timesteps_6': np.float64(54.68701868437718), 'mean_timesteps_7': np.float64(95.925), 'std_timesteps_7': np.float64(54.6872776104113), 'mean_timesteps_8': np.float64(96.9), 'std_timesteps_8': np.float64(58.66389698011629), 'mean_timesteps_9': np.float64(95.58000000000001), 'std_timesteps_9': np.float64(57.964850175505546), 'mean_timesteps_10